In [133]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from imblearn.pipeline import Pipeline as Pipel
from xgboost import XGBClassifier
import pickle

In [3]:
df = pd.read_csv("data_loan.csv") 
df.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [90]:
from sklearn.preprocessing import LabelEncoder
labelenmkt = LabelEncoder()

In [131]:
labelenmkt.classes_

array(['N', 'Y'], dtype=object)

In [92]:
df['Loan_Status'] = labelenmkt.fit_transform(df['Loan_Status'])

In [93]:
X=df.drop(columns='Loan_Status')
y = df['Loan_Status']

In [132]:
df.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,1
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,0
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,1
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,1
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,1


In [94]:
X_train, X_test, y_train, y_test = train_test_split(X,y, train_size=0.8)

In [95]:
y_train.value_counts()

1    345
0    146
Name: Loan_Status, dtype: int64

In [114]:
over = RandomOverSampler(sampling_strategy=1)
# 'minority'
# X_over, y_over=over.fit_resample(X_train,y_train)
# y_over.value_counts()

In [68]:
class DataframeFunctionTransformer():
    def __init__(self, func):
        self.func = func
    def transform(self, input_df, **transform_params):
        return self.func(input_df)

    def fit(self, X, y=None, **fit_params):
        return self
    def fit_transform(self, X, y=None):
        self.fit(X)
        return self.transform(X)

In [70]:
def pro_dataframe(input_df):
    input_df = input_df.age.dropna().reset_index(drop = True)
    return input_df

# pipeline = Pipeline([
#     ("uppercase", DataframeFunctionTransformer(pro_dataframe))
# ])
# pipeline.fit_transform(df)

In [98]:
numeric_transformer = SimpleImputer(strategy='constant', fill_value=999)
numeric_transformer2 = SimpleImputer(strategy='constant', fill_value=2.0)
# categorical_transformer = SimpleImputer(strategy='constant', fill_value='missing')
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

In [80]:
numeric_features = X.drop(columns='Credit_History')._get_numeric_data().columns
categorical_features = X.drop(columns='Loan_ID').select_dtypes(include=['object']).columns
preprocessor = ColumnTransformer(
    transformers=[('drop_columns', 'drop', ['Loan_ID']),
        ('num', numeric_transformer, numeric_features),
        ('num2', numeric_transformer2, ['Credit_History']),
        ('cat', categorical_transformer, categorical_features)],remainder='passthrough')

In [134]:
xgb = XGBClassifier()

In [99]:
rand = RandomForestClassifier(max_depth=7)

In [142]:
rf = Pipel(steps=[('over_sample', over),('preprocessor', preprocessor),
                      ('classifier', xgb)])
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)

In [126]:
X_test

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
338,LP002113,Female,No,3+,Not Graduate,No,1830,0.0,NaN,360.0,0.0,Urban
225,LP001751,Male,Yes,0,Graduate,No,3250,0.0,170.0,360.0,1.0,Rural
515,LP002659,Male,Yes,3+,Graduate,No,3466,3428.0,150.0,360.0,1.0,Rural
95,LP001326,Male,No,0,Graduate,NaN,6782,0.0,NaN,360.0,NaN,Urban
522,LP002692,Male,Yes,3+,Graduate,Yes,5677,1424.0,100.0,360.0,1.0,Rural
...,...,...,...,...,...,...,...,...,...,...,...,...
464,LP002493,Male,No,0,Graduate,No,4166,0.0,98.0,360.0,0.0,Semiurban
461,LP002484,Male,Yes,3+,Graduate,No,7740,0.0,128.0,180.0,1.0,Urban
206,LP001693,Female,No,0,Graduate,No,3244,0.0,80.0,360.0,1.0,Urban
7,LP001014,Male,Yes,3+,Graduate,No,3036,2504.0,158.0,360.0,0.0,Semiurban


In [143]:
print(metrics.roc_auc_score(y_test,y_pred))
print(metrics.classification_report(y_test,y_pred))
# metrics.plot_confusion_matrix(clf,X_test,y_test)

0.7284020327498588
              precision    recall  f1-score   support

           0       0.83      0.52      0.64        46
           1       0.77      0.94      0.84        77

    accuracy                           0.78       123
   macro avg       0.80      0.73      0.74       123
weighted avg       0.79      0.78      0.77       123



In [144]:
pickle.dump( rf, open( "mini_project4.p", "wb" ) )